In [53]:
%matplotlib inline
import pandas as pd
import numpy as np
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))# nrows=1048576)

In [54]:
data.head()

,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06


In [55]:
columns=['CountryCode']
countries = pd.read_csv("data/Country.csv", usecols=(columns))

In [56]:
countries.head()

,CountryCode
0,AFG
1,ALB
2,DZA
3,ASM
4,ADO


In [57]:
# find some measures that seem important and are often present, filter countries with all those intact, train network

In [58]:
columns=['LongDefinition', 'SeriesCode']
series = pd.read_csv("data/Series.csv", usecols=(columns))

In [59]:
series.head()

,SeriesCode,LongDefinition
0,BN.KLT.DINV.CD,Foreign direct investment are the net inflows ...
1,BX.KLT.DINV.WD.GD.ZS,Foreign direct investment are the net inflows ...
2,BX.KLT.DINV.CD.WD,Foreign direct investment refers to direct inv...
3,BM.KLT.DINV.GD.ZS,Foreign direct investment are the net inflows ...
4,BN.TRF.KOGT.CD,Net capital account records acquisitions and d...


In [60]:
print (len(countries))
print (len(series))

247
1345


In [61]:
#isUsedSeries = [(name in seriesToUse) for name in series.SeriesCode.values]
#filteredSeries = series[isUsedSeries]
#filteredSeriesNames = filteredSeries.SeriesCode.values

#useableData = [(name in filteredSeriesNames) for name in data.IndicatorCode.values]
#filteredData = data[useableData]
seriesCounts = data['IndicatorCode'].value_counts()

a = seriesCounts.to_dict()
useableData = [(a.get(name, 0) > 10000) for name in data.IndicatorCode.values]
filteredData = data[useableData]

In [62]:
useableSeries = [(a.get(name, 0) > 10000) for name in series.SeriesCode.values]
filteredSeries = series[useableSeries].SeriesCode.values

In [63]:
print(len(filteredData))
print(len(data))
print(len(filteredSeries))
print(len(series))

732948
5656458
63
1345


In [66]:
# a = data[(data.CountryCode == 'ARB') & ((data.IndicatorCode == 'SP.ADO.TFRT') | (data.IndicatorCode == 'SP.POP.DPND'))]
useableCountries = []
for country in countries.CountryCode.values:
    useable = True
    filterByCountry = filteredData[(data.CountryCode == country)]
    for seriesName in filteredSeries:
        if len(filterByCountry[(filterByCountry.IndicatorCode == seriesName)]) == 0:
            useable = False;
            break;
    if useable:
        useableCountries.append(country)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [67]:
len(useableCountries)

175

In [68]:
goodData = [(country in useableCountries) for country in filteredData.CountryCode.values]
useableData = filteredData[goodData]

In [69]:
print(len(useableData))
useableData.head()

559891


,CountryCode,IndicatorCode,Year,Value
350,EAP,SP.ADO.TFRT,1960,75.043631
351,EAP,SP.POP.DPND,1960,78.574185
352,EAP,SP.POP.DPND.OL,1960,6.480808
353,EAP,SP.POP.DPND.YG,1960,71.976235
359,EAP,SP.DYN.CBRT.IN,1960,26.635950


In [70]:
# Filtered Data reasonably above, need to:
# Seperate into few year blocks, with next year values as results to guess
# Train a few different models to guess next years values

In [101]:
predictionFeature = 'SP.ADO.TFRT'
yearsToPredict = 5
predictionData = countryData[(countryData.IndicatorCode == predictionFeature)]

trainingValues = []
trainingResults = []
for country in useableCountries:#[0:20]: # 213
    countryData = filteredData[(filteredData.CountryCode == country)]
    predictionCountryData = predictionData[(predictionData.CountryCode == country)]
    pastData_5 = []
    pastData_4 = []
    pastData_3 = []
    pastData_2 = []
    pastData_1 = []
    for year in range(1960, 2010): # 1960
        pastData_5 = pastData_4
        pastData_4 = pastData_3
        pastData_3 = pastData_2
        pastData_2 = pastData_1
        
        yearData = countryData[(countryData.Year == year)]
        
        pastData_1 = []
        for series in filteredSeries:#[0:30]: # 40
            # TODO: see if I can get rid of this loop
            seriesData = yearData[(yearData.IndicatorCode == series)].Value.values
            if len(seriesData) != 1:
                pastData_5 = []
                pastData_4 = []
                pastData_3 = []
                pastData_2 = []
                pastData_1 = []
                break;
            value = seriesData[0]
            #print (value)
            pastData_1.append(value)
            
        predictionValues = []
        for i in xrange(yearsToPredict):
            predictionValues.extend(predictionCountryData[(countryData.Year == year + 1 + i)].Value.values)
            
        valid = (len(predictionValues) == yearsToPredict) & (len(pastData_5) != 0) & (len(pastData_4) != 0) & (len(pastData_3) != 0) & (len(pastData_2) != 0) & (len(pastData_1) != 0)
        if valid:
            trainingValues.append(pastData_5 + pastData_4 + pastData_3 + pastData_2 + pastData_1)
            trainingResults.append(predictionValue[0])

In [1]:
xData = pd.DataFrame(trainingValues)
yData = pd.DataFrame(trainingResults)
print (len(xData))
print (len(yData))
xData.head()

NameError: name 'pd' is not defined

In [97]:
countryData = filteredData[(filteredData.CountryCode == 'AZE')]
countryData.head()

,CountryCode,IndicatorCode,Year,Value
4527,AZE,SP.ADO.TFRT,1960,43.221000
4528,AZE,SP.POP.DPND,1960,81.868653
4529,AZE,SP.POP.DPND.OL,1960,11.702121
4530,AZE,SP.POP.DPND.YG,1960,70.166533
4531,AZE,SP.DYN.CBRT.IN,1960,42.267000


In [100]:
yearData = countryData[(countryData.Year == 1965)]
yearData

,CountryCode,IndicatorCode,Year,Value
141970,AZE,SP.ADO.TFRT,1965,4.320600e+01
141971,AZE,SP.POP.DPND,1965,9.973410e+01
141972,AZE,SP.POP.DPND.OL,1965,1.129377e+01
141973,AZE,SP.POP.DPND.YG,1965,8.844033e+01
141974,AZE,SP.DYN.CBRT.IN,1965,3.720500e+01
141975,AZE,AG.PRD.CROP.XD,1965,6.240000e+01
141976,AZE,SP.DYN.CDRT.IN,1965,1.109800e+01
141977,AZE,SP.DYN.TFRT.IN,1965,5.352000e+00
141978,AZE,AG.PRD.FOOD.XD,1965,5.860000e+01
141979,AZE,AG.LND.TOTL.K2,1965,8.321700e+04


In [104]:
xData.to_csv("data/xSave.csv")
yData.to_csv("data/ySave.csv")